In [1]:
import json
import re

In [2]:
with open("database/1103output/ref_list.json","r") as f:
    ref_list=json.load(f)

In [3]:
with open("database/1103output/context_to_ref_id.json","r") as f:
    context_to_ref_id=json.load(f)

In [4]:
used_ref_list=list(set([r["ref_id"] for r in context_to_ref_id]))

In [5]:
abst_list=[r["abstract"] for r in ref_list if r["ref_id"] in used_ref_list]

In [16]:
concn_list=[r["main"][2]["text"] for r in ref_list if r["ref_id"] in used_ref_list if len(r["main"])>3]

In [17]:
concn_list[1]

'To inform source reduction of marine microdebris contamination based on ecological effects, field studies are required to examine the interactions between microdebris exposure and intake by marine organisms. Our results demonstrate widespread contamination of the central GBR WHA with marine microdebris, including contamination of surface waters and intake in a common, small coral reef fish. These findings corroborate previous studies reporting microdebris contamination in GBR (sub-)surface waters39,40 and reef organisms29,41, indicating that contamination is ubiquitous across the GBR WHA. Marine microdebris was found in all 22 surface water tows, and in 57 of the 60 Lemon damselfish examined, with fibres (86%) being much more common than particles (14%) among the total 1,002 items detected. High abundance of fibres relative to particles in surface waters and marine fish has been reported previously in some42,43, but not all7,44 microdebris studies. Our findings show that marine microd

In [14]:
def clean_text(text):
    pattern = r'\.(?=[A-Za-z])'
    text= re.sub(pattern, '. ', text)
    #連続したピリオドを削除
    text=re.sub(r'\.{2,}', '.', text)
    return text

def unify_text(text):
    text = text.replace("。", "\n")
    text = text.replace(".", "\n")
    text = text.replace("．", "\n")
    text = text.replace("？", "\n")
    text = text.replace("?", "\n")
    text = text.replace("！", "\n")
    text = text.replace("!", "\n")
    text = text.replace("；", "\n")
    text = text.replace(";", "\n")
    text = text.replace("：", "\n")
    text = text.replace(":", "\n")
    text = text.replace("　", " ")

    return text

def split_text(text, chunk_size_limit=2000):
    text = unify_text(text)

    text_list = text.split("\n")

    chunk_text_list = []
    temp_text = ""
    for t in text_list:
        t = t.strip()
        temp_text += t+"."
        if len(temp_text) < chunk_size_limit:
            continue
        else:
            chunk_text_list.append(temp_text)
            temp_text = ""

    chunk_text_list.append(temp_text)
    return chunk_text_list

In [18]:

chunk_text_list=[]
for c in concn_list:
    chunk_text_list+=split_text(c)

chunk_text_list=[clean_text(i) for i in chunk_text_list if len(i)>10]
len(chunk_text_list)

46837

In [19]:
with open("database/1103output/concn_list.json","w") as f:
    json.dump(chunk_text_list,f,indent=4)

In [45]:
chunk_text_list=[]
for abst in abst_list:
    chunk_text_list+=split_text(abst)

chunk_text_list=[clean_text(i) for i in chunk_text_list if len(i)>10]

In [47]:
with open("database/1103output/abst_list.json","w") as f:
    json.dump(chunk_text_list,f,indent=4)

In [48]:
all_intro_list=[r["main"][0]["text"] for r in ref_list]

In [49]:
chunk_intro_list=[]
for text in all_intro_list:
    chunk_intro_list+=split_text(text)

chunk_intro_list=[clean_text(i) for i in chunk_intro_list if len(i)>10]


In [50]:
len(chunk_intro_list),chunk_intro_list[0][-10:]

(173802, 'e de novo.')

In [51]:
with open("database/1103output/all_intro_list.json","w") as f:
    json.dump(chunk_intro_list,f,indent=4)